#Data Collection



In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

data = pd.read_csv('/content/MSFT.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
data.head(10)

# Data Preprocessing

In [15]:
data.fillna(method='ffill', inplace=True)

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# Feature Engineering

In [16]:
sequence_length = 14
X, y = [], []
for i in range(sequence_length, len(scaled_data)):
    X.append(scaled_data[i - sequence_length:i])
    y.append(scaled_data[i, 3])  # Using 'close price' as the target

X, y = np.array(X), np.array(y)

# Model Selection

In [17]:
train_size = int(0.8 * len(X))
X_train, X_test, y_train, y_test = X[:train_size], X[train_size:], y[:train_size], y[train_size:]

In [ ]:
X_train.shape

In [ ]:
X_train

# Model Training

In [ ]:
model = Sequential()
model.add(LSTM(200, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(200, return_sequences=False))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, batch_size=64, epochs=50, verbose=1)

# Evaluation

In [ ]:
y_pred = model.predict(X_test)

y_pred = scaler.inverse_transform(np.hstack((np.zeros((len(y_pred), scaled_data.shape[1] - 1)), y_pred)))
y_test = scaler.inverse_transform(np.hstack((np.zeros((len(y_test), scaled_data.shape[1] - 1)), np.expand_dims(y_test, axis=1))))

y_pred_close = y_pred[:, -1]
y_test_close = y_test[:, -1]

mae = mean_absolute_error(y_test_close, y_pred_close)
rmse = np.sqrt(mean_squared_error(y_test_close, y_pred_close))

print(f"Mean Absolute Error: {mae}")
print(f"Root Mean Squared Error: {rmse}")

# Visualize the results
plt.figure(figsize=(12, 6))
plt.plot(data.index[-len(y_test):], y_test_close, label='True Close Price', color='blue')
plt.plot(data.index[-len(y_test):], y_pred_close, label='Predicted Close Price', color='red')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.title('Stock Price Prediction')
plt.legend()
plt.show()